In [1]:
import os
from pathlib import Path
import pandas as pd

In [56]:
try:
  from qdrant_client import models, QdrantClient
except:
  !pip install qdrant-client > /dev/null
  from qdrant_client import models, QdrantClient
try:
  from sentence_transformers import SentenceTransformer
except:
  !pip install sentence_transformers > /dev/null
  from sentence_transformers import SentenceTransformer

In [4]:
# Hiding warnings
import warnings
warnings.filterwarnings('ignore')

Helper functions

In [46]:
def search_metrics(client, encoder, metric, cat):
    category_result = []

    collection_name = f"citp_{metric}"

    hits = client.search(
        collection_name=collection_name,
        query_vector=encoder.encode(cat).tolist(),
        limit=3
    )
    for hit in hits:
        category_result.append({
            **hit.payload,
            "score": hit.score,
            "name": cat
        })

    result_df = pd.DataFrame(category_result)

    highest = rename_columns(highest_score(result_df), f"{metric}_highest")
    simple = rename_columns(simple_count(result_df), f"{metric}_simple")
    weighted = rename_columns(weighted_count(result_df), f"{metric}_weighted")

    long_df = pd.concat([highest, simple, weighted])

    return long_df


def rename_columns(df: pd.DataFrame, suffix:str) -> pd.DataFrame:
    """Function to add a suffix to specific columns
    """
    df = df.rename(columns={
        "code": f"code_{suffix}",
        "score": f"score_{suffix}",
        "count": f"score_{suffix}"})
    return df


def weighted_count(df: pd.DataFrame) -> pd.DataFrame:
    """Function to select the classification with the highest count
    of classification weighted by scores
    """
    grouped = df.groupby(['label', 'citp_code']).sum("score").reset_index()
    result = grouped.loc[grouped.groupby('label')['score'].idxmax()]
    return result


def simple_count(df: pd.DataFrame) -> pd.DataFrame:
    """Function to select the classification with the highest count
    of classification
    """
    counted = df.groupby(['label', 'citp_code']).size().reset_index(name='count')
    # Find the 'coicop_code' with the highest count for each 'name'
    result = counted.loc[counted.groupby('label')['count'].idxmax()]
    return result


def highest_score(df: pd.DataFrame) -> pd.DataFrame:
    """Function to select the classification with the highest score
    """
    # Find the index of the maximum score for each name
    idx = df.groupby('label')['score'].idxmax()

    # Select the rows corresponding to these indices
    result = df.loc[idx]
    return result


def majority_vote(df: pd.DataFrame) -> pd.DataFrame:
    counted = df.groupby(['label', 'citp_code']).size().reset_index(name='count')
    # Find the 'coicop_code' with the highest count for each 'name'
    result = counted.loc[counted.groupby('label')['count'].idxmax()]

    result["confidence"] = result["count"] / 6

    return result.to_dict(orient='records')[0]

## Load data

In [5]:
home = Path('C:/Users/f.migone/Desktop/codif_rag_ene')

In [6]:
examples_df = pd.read_excel(home.joinpath("data_T4_2024_enem.xlsx"), dtype=str)
products_df = pd.read_excel(home.joinpath("file_manually_coded.xlsx"), dtype=str)

Renaming columns for convenience

In [7]:
examples_df.columns = ["label", "citp_code"]

In [8]:
products_df = products_df[["label", "code"]]
products_df.columns = ["label", "citp_code"]

In [9]:
products_df['citp_code'].unique()

array(['422005', '611001', '332003', '333202', '532002', '911002',
       '541401', '721001', '331101', '333104', '342204', '613001',
       '613098', '921001', '712002', '612005', '711006', '216299',
       '532004', '931201', '522001', '962012', '941001', '521106',
       '314199', '514001', '723001', '752001', '265202', '421004',
       '321204', '321205', '322002', '611007', '961005', '516005',
       '751202', '832103', '521101', '523001', '962013', '711004',
       '833003', '832203', '832201', '832101', '833002', '314106',
       '711001', '313099', '522003', '521102', '521104', '521199',
       '521108', '521105', '962010', '521107', '323001', '524201',
       '834002', '752006', '834001', '235102', '314102', '962099',
       '933003', '753304', '753102', '512000', '611006', '933004',
       '325199', '342205', '130005', '130099', '962004', '531002',
       '431204', '235103', '741001', '612002', '612001', '911001',
       '234001', '233005', '234003', '235299', '514003', '7540

In [10]:
examples_df['citp_code'].unique()

array(['021004', '031006', '031007', '111206', '111403', '111405',
       '111407', '111408', '112002', '112005', '120008', '130001',
       '130008', '130012', '130099', '140002', '140004', '140005',
       '213201', '213208', '214005', '214099', '215003', '216101',
       '216103', '216203', '221001', '222001', '222002', '224099',
       '225000', '226001', '226006', '226099', '231002', '232000',
       '233003', '233004', '233005', '234001', '234002', '235101',
       '235102', '235103', '235201', '235202', '235299', '235399',
       '235402', '235499', '241001', '241003', '241099', '242005',
       '242006', '242008', '243099', '252001', '263302', '263304',
       '263305', '263307', '264203', '265101', '265201', '265202',
       '311101', '311201', '311301', '311302', '311303', '311304',
       '311499', '312004', '313001', '314101', '314102', '314104',
       '314105', '314106', '314107', '314108', '314109', '314302',
       '314304', '315104', '321102', '322001', '322002', '3220

There are several products in the "ivory_cost_09032024_valide_code.xlsx" files which belongs to COICOP codes not present in the example file. Those products may not be classified correctly, because there is no example for the subclass they belong to. In order to get a reasonable idea of the accuracy of the proposed methodology, we remove those products (if we populate the database with all the already classified products, we should have examples for all subclassess).

In [11]:
products_df = products_df[products_df['citp_code'].isin(examples_df['citp_code'].unique())]

## Qdrant database

Load data from products already classified in the Qdrant Database.

In [12]:
data_dict = examples_df.to_dict("records")

In [ ]:
encoder = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\f.migone\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\f.migone\.cache\huggingface\hub\models--sentence-transformers--paraphrase-xlm-r-multilingual-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [54]:
client = QdrantClient(":memory:")

### Cosine distance

In [57]:
client.create_collection(
    collection_name="citp_cosine",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [58]:
client.upload_points(
    collection_name="citp_cosine",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["label"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(data_dict)
    ],
)

### Dot product distance

In [59]:
client.create_collection(
    collection_name="citp_dot",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.DOT,
    ),
)

True

In [60]:
client.upload_points(
    collection_name="citp_dot",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["label"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(data_dict)
    ],
)

In [61]:
def predict(input_dict, client, encoder):
    """
    Prediction given the request input
    :param input_dict: [dict], product to be classfied
    :param client: [QdrantClient], Qdrant client
    :param encoder: [SentenceTransformer], SentenceTransformer model for embeddings
    :return: [dict], prediction
    """

    name = input_dict.get("label")


    results_cosine = search_metrics(client, encoder, "cosine", name)
    results_dot = search_metrics(client, encoder, "dot", name)

    results_concat = pd.concat([results_cosine, results_dot])

    result = majority_vote(results_concat)

    result = {
        "ground_truth": input_dict.get("citp_code"),
        "name": name,
        "classification": result.get("citp_code"),
        "confidence": result.get("confidence")
    }

    return result

## Predictions

In [62]:
query_dict = products_df.to_dict("records")

In [63]:
query_dict

[{'label': 'accueillir les requérants et les faire enrôler  enrôler les requérants',
  'citp_code': '422005'},
 {'label': 'acheteur de cacao  cacao', 'citp_code': '611001'},
 {'label': 'acheteur de cacao  peser, ramassager', 'citp_code': '611001'},
 {'label': 'acheteur de produits agricoles (anacarde et mais)  acheter les produits agricoles (anacarde, mais) et revendre',
  'citp_code': '332003'},
 {'label': "acheteur d'hévéa  peser et acheter hevea", 'citp_code': '332003'},
 {'label': 'agent commercial  promotion de produit (savon et chocolat et autre...)',
  'citp_code': '333202'},
 {'label': 'agent commercial  service les clients et faire la reception',
  'citp_code': '333202'},
 {'label': 'agent commerciale cie  accueillir les clients',
  'citp_code': '333202'},
 {'label': 'agent de mairie  ramaser les ordures', 'citp_code': '911002'},
 {'label': "agent de securite  surveillance de l'universite de bouake",
  'citp_code': '541401'},
 {'label': 'agent de securite  vigiles', 'citp_code

In [64]:
%%time
results = [predict(item, client=client, encoder=encoder) for item in query_dict]

C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(
C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` ins

CPU times: total: 45min 7s
Wall time: 31min 52s


C:\Users\f.migone\AppData\Local\Temp\ipykernel_17152\478697674.py:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


In [65]:
results_df = pd.DataFrame(results)
results_df["correct"] = results_df["ground_truth"] == results_df["classification"]
results_df.to_csv(home.joinpath("results.csv"), index=False)

In [66]:
accuracy = results_df["correct"].sum() / len(results_df)

In [67]:
print("Accuracy: {:.2f}%\nExample DB: {}\nProducts classfied:{}".format(accuracy * 100, len(data_dict), len(query_dict)))

Accuracy: 50.27%
Example DB: 5957
Products classfied:3280
